In [57]:
import numpy as np
import cv2
import csv
import pandas as pd
from glob import glob
import xml.etree.ElementTree as xet

from shutil import copy

In [58]:
import os

In [59]:
rf = pd.read_csv('labels.csv')
rf.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [60]:
# Labeling in yolo
#    height, width, center_x, center_y

In [61]:
def chkng(path):

    chkng = xet.parse(path).getroot()
    name = chkng.find('filename').text
    filename = f'./images/{name}'
    chkng_size = chkng.find('size')
    #width
    width = int(chkng_size.find('width').text)
    height = int(chkng_size.find('height').text)
    return filename, width, height
    
rf[['filename','width','height']] = rf['filepath'].apply(chkng).apply(pd.Series)


In [62]:
rf.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height
0,./images\N1.xml,1093,1396,645,727,./images/N1.jpeg,1920,1080
1,./images\N100.xml,134,301,312,350,./images/N100.jpeg,450,417
2,./images\N101.xml,31,139,128,161,./images/N101.jpeg,249,239
3,./images\N102.xml,164,316,216,243,./images/N102.jpeg,478,395
4,./images\N103.xml,813,1067,665,724,./images/N103.jpeg,1800,1200


In [63]:
rf['center_x'] = (rf['xmax'] + rf['xmin'])/ (2* rf['width'])
rf['center_y'] = (rf['ymax'] + rf['ymin'])/ (2* rf['height'])
rf['bb_width'] = (rf['xmax'] - rf['xmin'])/rf['width']
rf['bb_height'] = (rf['ymax'] - rf['ymin'])/rf['height']

In [64]:
rf.head()

,filepath,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images\N1.xml,1093,1396,645,727,./images/N1.jpeg,1920,1080,0.648177,0.635185,0.157812,0.075926
1,./images\N100.xml,134,301,312,350,./images/N100.jpeg,450,417,0.483333,0.793765,0.371111,0.091127
2,./images\N101.xml,31,139,128,161,./images/N101.jpeg,249,239,0.341365,0.604603,0.433735,0.138075
3,./images\N102.xml,164,316,216,243,./images/N102.jpeg,478,395,0.502092,0.581013,0.317992,0.068354
4,./images\N103.xml,813,1067,665,724,./images/N103.jpeg,1800,1200,0.522222,0.578750,0.141111,0.049167


In [65]:
#splitting data for training and testing

rf_train = rf.iloc[:200]
rf_test = rf.iloc[200:]

In [66]:
#text ID must consist of class_id, centery, centerx, bb_width, bb_height

In [67]:
fileNameTrain = './data_images/train'
values = rf_train[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h in values:
    nameImage = os.path.split(fname)[-1]
    textName = os.path.splitext(nameImage)[0]
    pathImage = os.path.join(fileNameTrain, nameImage)
    fileLabel = os.path.join(fileNameTrain, textName + '.txt')
# copying the images into this folder
    copy(fname, pathImage)
#generating a text file with its labels in it.
    labelling = f'0 {x} {y} {w} {h}'
    with open(fileLabel, mode = 'w') as f:
        f.write(labelling)
        f.close()

In [68]:
fileNameTest = './data_images/test'
values = rf_test[['filename','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h in values:
    nameImage = os.path.split(fname)[-1]
    textName = os.path.splitext(nameImage)[0]
    pathImage = os.path.join(fileNameTest, nameImage)
    fileLabel = os.path.join(fileNameTest, textName + '.txt')
# copying the images into this folder
    copy(fname, pathImage)
#generating a text file with its labels in it.
    labelling = f'0 {x} {y} {w} {h}'
    with open(fileLabel, mode = 'w') as f:
        f.write(labelling)
        f.close()